# 1. DATA PREP

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import random
import altair as alt
import graphviz

from sklearn.feature_extraction import FeatureHasher
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [3]:
pd.set_option("max_columns", 100)
pd.set_option("max_rows", 100)

In [4]:
data_train = pd.read_csv("data_train.csv")
target_train = pd.read_csv("target_train.csv")
data_test=pd.read_csv("data_test.csv")

In [6]:
target_test = pd.read_csv("target_test.csv").drop('Unnamed: 0', axis=1).set_index('index')

data_test['index'] = data_test['LAPISSUGL'].astype(str) + '-' + data_test['ARUSKOD'].astype(str)
data_test = data_test.set_index('index')

data_train['index'] = data_train['LAPISSUGL'].astype(str) + '-' + data_train['ARUSKOD'].astype(str)
data_train = data_train.set_index('index')

target_train['index'] = data_train.index
target_train = target_train.set_index('index')

## 1.1 Numeric, categorical and date type variables

In [7]:
column_list = data_train.columns

categorical_variables = [
    "ARUSKOD",
    "ARUSMEGYE",
    "ARUSTI1",
    "ARUSTI2",
    "ARUSTI3",
    "ARUSTI4",
    "ZVEVOCSAT",
    "CCHANNEL",
    "CNETWORK",
    "CNETTYPE",
    "bevasarlokozpont",
    "egyiksincsakozelb",
    "felsooktatasiintezm",
    "furdo",
    "hotel",
    "irodahazhivatal",
    "korhazrendelo",
    "kosarlabdapalya",
    "kozepiskola",
    "mozi",
    "muzeum",
    "muvelodesikozpont",
    "ovoda",
    "parkjatszoter",
    "piac",
    "posta",
    "sportcsarnoksportpa",
    "sportletesitmeny",
    "stadioncsarnok",
    "strand",
    "szinhaz",
    "temeto",
    "templom",
    "teniszpalya",
    "uszoda",
    "udulo",
    "wellnessfitness",
    "LAPISSUGL",
    "LAPMEDTER",
    "LAPTERMCS",
    "LAPKIADO",
    "LAPCS1",
    "LAPCS2",
    "LAPPER1",
    "LAPPER2",
    "LAPFORMAT",
]

numeric_variables = [
    "Z_LAPFELE",
    "LAPARBRUT",
    "LAPARNET",
    "LAPMEGJSZ",
    "NAPOK_POLCON",
    "ELOZO_NAPOK_POLCON",
    #"NAPI_ELADOTT_DB",
    "ELOZO_NAPI_ELADOTT_DB",
]

date_vars = [
    "KFDELDATE",
]

len(categorical_variables+numeric_variables+date_vars)
print(len(categorical_variables) == len(set(categorical_variables)), ": ckeck for duplicate")
print(len(numeric_variables) == len(set(numeric_variables)), ": ckeck for duplicate")
seen = set()
dupes = [x for x in categorical_variables if x in seen or seen.add(x)]
dupes
# kétszer szerepeltek: ['LAPISSUGL', 'ARUSMEGYE', 'CNETTYPE']
set(column_list) - set(categorical_variables + numeric_variables + date_vars) #kimaradt: date

True : ckeck for duplicate
True : ckeck for duplicate


{'LAPMELL1',
 'LAPMELL10',
 'LAPMELL2',
 'LAPMELL3',
 'LAPMELL4',
 'LAPMELL5',
 'LAPMELL6',
 'LAPMELL7',
 'LAPMELL8',
 'LAPMELL9'}

In [8]:
data_train = data_train.join(target_train)

for col in numeric_variables:
    data_train[col] = data_train[col].astype('float32')
    data_test[col] = data_test[col].astype('float32')
    
for col in categorical_variables:
    data_train[col] = data_train[col].astype("object")
    data_test[col] = data_test[col].astype("object")

In [9]:
data_train.NAPI_ELADOTT_DB = data_train.NAPI_ELADOTT_DB.astype('float32')

* van 1 outlier a date-ben, kidobom azt a megfigyelést, aztán datetime-ot csinálok belőle

In [10]:
data_train.KFDELDATE.max()

99991231

In [11]:
data_train = data_train.loc[data_train.KFDELDATE<20220201]
data_train['KFDELDATE'] = pd.to_datetime(data_train['KFDELDATE'], format='%Y%m%d')
data_test['KFDELDATE'] = pd.to_datetime(data_test['KFDELDATE'], format='%Y%m%d')

In [12]:
#data_pp.dtypes #kúl minden

## 1.2 NA CHECK
* A LAPMELL VÁLTOZÓKAT KI KELL DOBNI, MERT NINCS BENNÜK INFO
* ÍGY **NEM MARAD MISSING A DATA-BAN**

In [13]:
#data_test.isna().mean().sort_values()
#print([df.LAPMELL1.value_counts()]) # a többi lapmell üres.. -> egyértelmű drop és csá

In [14]:
data_train = data_train.loc[:, ~data_train.columns.str.contains("^LAPMELL", case=False)]
data_test = data_test.loc[:, ~data_test.columns.str.contains("^LAPMELL", case=False)]

## 1.3 SET INDEX

In [15]:
data_train["index"] = data_train["LAPISSUGL"].astype(str) + '-' + data_train["ARUSKOD"].astype(str)
data_train = data_train.set_index("index")

data_test["index"] = data_test["LAPISSUGL"].astype(str) + '-' + data_test["ARUSKOD"].astype(str)
data_test = data_test.set_index("index")

## 1.4 EXPLORE

**MIT FIGYELÜNK MEG AZ ADATBAN?**

* LAPISSUGL VÁLTOZÓ JELENTÉSE:
    * DÁTUM ÉS LAPKIADO KOMBINÁCIÓJA
    * (A LAPKIADÓK EGYFÉLE FORMÁTUMOT HASZNÁLNAK)
* 10 ÁRUSKÓD, AZAZ ÁRUS VAN

**AZT HOGY A 2015.01.08-tól 2022.01.13-ig TARTÓ IDŐSZAKBAN NAPONTA MENNYIT ADOTT EL A 10 ÁRUS A KÜLÖNBÖZŐ LAPKIADÓKNÁL**

---

**MI A NAPI ELADOTT DB?**
* EZ VALAMI HÁNYADOSNAK TŰNIK, DE JÓ LENNE TUDNI A PONTOS TARTALMÁT, ANÉLKÜL NEHÉZ ELDÖNTENI MILYEN VÁLTOZÓKAT HASZNÁLHATOK


---

* 8  ÁRUSMEGYE, AZAZ A 10 ÁRUS 8 MEGYÉBEN TEVÉKENYKEDIK
    * VAN 2 MEGYE, AMIKBEN 2 ÁRUS VAN JELEN, A TÖBBIBEN 1. 
    * ERRE LEHETNE CSINÁLNI EGY VÁLTOZÓT
* MINDEGYIK ÁRUS 3 FÉLE LAPFORMÁTUMOT HASZNÁL
    * A LAPFORMÁTUM KIADÓ SPECIFIKUS
    * AZAZ A KIADÓ MELLETT NINCS ÉRTELME HASZNÁLNI A FROMÁTUMOT IS

* FOLYTONOS VÁLTOZÓK KORRELÁCIÓJA A FÜGGŐ VÁLTOZÓVAL

In [16]:
data_train[data_train.columns[1:]].corr()['NAPI_ELADOTT_DB'][:-1].sort_values()

NAPOK_POLCON            -0.313630
ELOZO_NAPOK_POLCON      -0.303270
LAPMEGJSZ               -0.126513
LAPARNET                -0.092764
LAPARBRUT               -0.092764
Z_LAPFELE                0.111490
ELOZO_NAPI_ELADOTT_DB    0.698211
Name: NAPI_ELADOTT_DB, dtype: float64

# 2. TRAIN-TEST SPLIT

In [17]:
df = None
def train_test_split_func(y_colname="NAPI_ELADOTT_DB", df=data_train):
    retlist = train_test_split(
        df.drop(y_colname, axis=1).values,
        df[y_colname].values,
        test_size=0.2,
        random_state=42,
    )

    return [
        pd.DataFrame(f, columns=[f for f in df.columns if not f == y_colname])
        if i < 2
        else pd.DataFrame(f, columns=[y_colname])
        for i, f in enumerate(retlist)
    ]

In [18]:
X_train, X_test, y_train, y_test = train_test_split_func(df=data_train)

# 3. FEATURE ENGINEERING

* __[fit-transform]("https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe")__
* __[sklearn preprocessing]("https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features")__
* __[Ordinal encoder]("https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html")__

### A) TRAIN

In [19]:
def create_encoded_column(
    encoder,
    colname_s,
    encode_string,
    X_train=X_train,
    X_test=X_test,
    data_test=data_test,
):

    """
    Use encoder to fit_transform the list of columns in "colname_s" and insert the encoded column values
    to `x_colname` col in both train and test sets. Only fits on train, transforms test.
    """

    for variable in colname_s:
        """
        Fit label encoder on training data, and return encoded labels.
        """
        X_train[f"{variable}_{encode_string}"] = encoder.fit_transform(
            X_train[variable].values.reshape(-1, 1)
        )
        """
        Transform labels to normalized encoding on the test data
        """
        X_test[f"{variable}_{encode_string}"] = encoder.transform(
            X_test[variable].values.reshape(-1, 1)
        )
        data_test[f"{variable}_{encode_string}"] = encoder.transform(
            data_test[variable].values.reshape(-1, 1)
        )
        
    return X_train, X_test, data_test

In [20]:
# ENCODERs
oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)

In [21]:
# CREATE ENCODED COLUMNS
X_train, X_test, data_test = create_encoded_column(oe, categorical_variables, "oe")

In [22]:
# DROP ORIGINAL ONES
X_train = X_train.drop(categorical_variables, axis=1)
X_train.columns = X_train.columns.str.replace("_oe", "")

X_test = X_test.drop(categorical_variables, axis=1)
X_test.columns = X_test.columns.str.replace("_oe", "")

data_test = data_test.drop(categorical_variables, axis=1)
data_test.columns = data_test.columns.str.replace("_oe", "")

In [23]:
for col in categorical_variables:
    X_train[col] = X_train[col].astype('float32')
    X_test[col] = X_test[col].astype('float32')
    data_test[col] = data_test[col].astype('float32')

# 4. TREE
* __[DecisionTreeRegressor]("https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor.fit")__

In [24]:
use_cols = [
    "NAPOK_POLCON",
    "ELOZO_NAPOK_POLCON",
    "ELOZO_NAPI_ELADOTT_DB",
    "Z_LAPFELE",
    "LAPMEGJSZ",
    "LAPARBRUT",
    "LAPARNET",
    "ARUSMEGYE",
    "ARUSTI1",
    "ARUSTI2",
    "ARUSTI3",
    "ARUSTI4",
    "ZVEVOCSAT",
    "CCHANNEL",
    "CNETWORK",
    "CNETTYPE",
    "bevasarlokozpont",
    "egyiksincsakozelb",
    "felsooktatasiintezm",
    "furdo",
    "hotel",
    "irodahazhivatal",
    "korhazrendelo",
    "kosarlabdapalya",
    "kozepiskola",
    "mozi",
    "muzeum",
    "muvelodesikozpont",
    "ovoda",
    "parkjatszoter",
    "piac",
    "posta",
    "sportcsarnoksportpa",
    "sportletesitmeny",
    "stadioncsarnok",
    "strand",
    "szinhaz",
    "temeto",
    "templom",
    "teniszpalya",
    "uszoda",
    "udulo",
    "wellnessfitness",
    "LAPMEDTER",
    "LAPTERMCS",
    "LAPKIADO",
    "LAPCS1",
    "LAPCS2",
    "LAPPER1",
    "LAPPER2",
    "LAPFORMAT",
]

### A) TRAIN

In [39]:
def visualize_dec_tree(decision_tree):
    """
    Returns visualization of a decision tree
    """
    dot_data = tree.export_graphviz(
        decision_tree, out_file=None, feature_names=decision_tree.feature_names_in_, filled=True
    )
    return graphviz.Source(dot_data, format="png")


def dec_tree_rmse(
    params, x_colnames, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
):
    """
    Fit decision tree with params, and returns rmse and plot if plot.
    """
    
    dec_tree = DecisionTreeRegressor(**params).fit(X_train.loc[:, x_colnames], y_train)
    test_preds = dec_tree.predict(X_test.loc[:, x_colnames])
    rmse = mean_squared_error(y_true=y_test, y_pred=test_preds, squared=False)

    source = pd.DataFrame(
        {"y": y_test["y_true"], "y_pred": test_preds, #"x_colnames": X_test[x_colnames]
        }
    )

    return rmse, source, dec_tree

In [26]:
params = {"max_depth": 10}
rmse, source_oe, dec_tree = dec_tree_rmse(params=params, x_colnames=use_cols)

In [27]:
rmse

0.1311836884229609

### B) TEST

In [40]:
def dec_tree_rmse_test(
    params, x_colnames, X_train=X_train, X_test=data_test, y_train=y_train
):
    """
    Fit decision tree with params, and returns rmse and plot if plot.
    """

    dec_tree = DecisionTreeRegressor(**params).fit(X_train.loc[:, x_colnames], y_train)
    test_preds = dec_tree.predict(X_test.loc[:, x_colnames])
    rmse = mean_squared_error(y_true=y_test, y_pred=test_preds, squared=False)

    source = pd.DataFrame(
        {"id": X_test.index, "y_pred": test_preds,}  # "x_colnames": X_test[x_colnames]
    )

    return source, dec_tree

In [41]:
target_test

,y_true
index,
10679220020200006-531118,0.093750
10754860020170014-614051,0.266667
10414850020160317-917085,0.027778
10534000020150002-531118,0.068966
10534000020170001-535460,0.090909
...,...
10554880020150022-917085,0.142857
10754860020190015-101223,0.200000
10754860020170023-823031,0.666667


In [43]:
params = {"max_depth": 10}
rmse, source_oe, dec_tree = dec_tree_rmse(params=params, x_colnames=use_cols, X_train=X_train, y_train=y_train, X_test=data_test, y_test=target_test)

In [44]:
# szerintem rossz adatra predikciót adtál le
rmse

0.14293928931693312

In [30]:
source_oe.columns = ["index","pred"]
source_oe.to_csv("prediction_df.csv")

In [31]:
source_oe

,index,pred
0,10679220020200006-531118,0.048686
1,10754860020170014-614051,0.132897
2,10414850020160317-917085,0.012733
3,10534000020150002-531118,0.025377
4,10534000020170001-535460,0.050717
...,...,...
1804,10554880020150022-917085,0.142857
1805,10754860020190015-101223,0.207359
1806,10754860020170023-823031,1.266667
1807,10754860020150023-516745,0.150000
